    Exnode 10. Text Transrator
        1. package loading
        2. data loading : 
            - reading 
            - drop & slice 
            - add
        3. text vocabulary ( vocabulary ... incoding(text sequence > num sequence))
            - english
            - french
            - vocabulary length > 변수         
        4. 전처리 
            - word max len
            - 모델 decoder 생성(french 2개 필요
            - padding 
            - one-hot incoding
            - validation data creation 
        5. model training 
            - package loading
            - incoder model ... hidden state , cell state creation 
            - decoder model .. input
            - decoder model .. output
            - model define & compile
            - model fit
        6. test model 생성
            - encoder model
            - decoder model
            - output layer design
            - word> num , num > word dictionary
            - 모델 진행
            - 문장 sample 입력 및 test
        7. 결론 
            - 모델 학습 전 전처리 부분에 많은 시간과 노력이 요구되었으며, 앞으로 진행하게 될 모든 학습에 전처리부문이 중요하게
              진행될 것으로 예상됩니다.
            - 모델학습등 모든 부분이 에러없이 진행되었으나 마지막 문장 해석부분에서 오류가 발생하였습니다. 
              5일간 고생하다가 node안에 고민하다 풀리지 않을 시 보라는 문장 예시를 보고 겨우 진행하였습니다. 
              이것을 하면서 심한 자괴감을 느꼈습니다.
              그동안 node를 보면서 진행하는 수준으로 느꼈고, 지금까지의 학습이 소용이 없었다는 것에 2달동안 진행한 것이
              겉핧기로 배운 것으로 생각되었습니다.
              좀 더 모델 진행 및 차원 변화등에 많은 지식을 쌓아야겠습니다. 
            

In [146]:
#1. package loading
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np
import unicodedata
import re
import os


In [147]:
#2. data loading -- read 
file_path = os.getenv('HOME')+'/aiffel/translator_seq2seq/data/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print('전체 샘플의 수 :',len(lines))
lines.sample(5) #샘플 5개 출력

전체 샘플의 수 : 197463


,eng,fra,cc
175437,I have an older brother and a younger sister.,J'ai un grand frère et une petite sœur.,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
163234,Muslims always pray facing toward Mecca.,Les musulmans prient toujours en faisant face ...,CC-BY 2.0 (France) Attribution: tatoeba.org #7...
96829,It was a strange experience.,C'était une expérience étrange.,CC-BY 2.0 (France) Attribution: tatoeba.org #6...
48048,Do I look like I care?,Ai-je l'air de m'en soucier ?,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
104861,Please sit down for a moment.,Veuillez vous asseoir un moment.,CC-BY 2.0 (France) Attribution: tatoeba.org #5...


In [148]:
#2. data loading -- drop & slice
lines = lines[['eng', 'fra']][:33000] # 3.3만개 샘플 사용
lines.sample(5)

,eng,fra
21249,Tom painted that.,Tom a peint cela.
25267,Keep away from me.,Gardez vos distances.
11779,Tell us a joke.,Raconte-nous une blague.
6564,You're silly.,Vous êtes idiotes.
447,Aim high.,Vise haut.


In [149]:
#2. data loading --- add
# 시작 토큰과 종료 토큰 추가
sos_token = '<sos>'
eos_token = '<eos>'
lines.fra = lines.fra.apply(lambda x : '<sos> '+ x + ' <eos>')
print('전체 샘플의 수 :',len(lines))
lines.sample(5)


전체 샘플의 수 : 33000


,eng,fra
9823,Do it this way.,<sos> Faites-le de cette manière. <eos>
25437,My hands are cold.,<sos> J'ai les mains gelées. <eos>
23928,I hate rainy days.,<sos> Je déteste les journées pluvieuses. <eos>
21192,Tom left earlier.,<sos> Tom est parti tout à l'heure. <eos>
2159,I'm skinny.,<sos> Je suis maigrichon. <eos>


In [150]:
# 프랑스어 악센트(accent) 삭제
def to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                   if unicodedata.category(c) != 'Mn')

# 단어와 구두점 사이에 공백 추가.  # 악센트 제거 함수 호출
def preprocess_sentence(sent):
    sent = to_ascii(sent.lower())
    sent = re.sub(r"([?.!,¿])", r" \1", sent)

  # (a-z, A-Z, ".", "?", "!", ",") 이들을 제외하고는 전부 공백으로 변환.
    sent = re.sub(r"[^a-zA-Z!.?]+", r" ", sent)
    sent = re.sub(r"sos+", r"<sos>", sent)
    sent = re.sub(r"eos+", r"<eos>", sent)
    
  # 다수 개의 공백을 하나의 공백으로 치환
    sent = re.sub(r"\s+", " ", sent)
    
    return sent

In [151]:
def load_preprocessed_data():
    encoder_input, decoder_input, decoder_target = [], [], []
   
    for i in range(len(lines)):
             
        # source 데이터 전처리
        src_line = [w for w in preprocess_sentence(lines.eng[i]).split()]
    
        # target 데이터 전처리
        tar_line = [w for w in preprocess_sentence(lines.fra[i]).split()]
      
        encoder_input.append(src_line)
        decoder_input.append(tar_line)
       
    return encoder_input, decoder_input

In [152]:
encoder, decoder = load_preprocessed_data()
print('인코더의 입력 :',encoder[:5])
print('디코더의 입력 :',decoder[:5])

인코더의 입력 : [['go', '.'], ['go', '.'], ['go', '.'], ['go', '.'], ['hi', '.']]
디코더의 입력 : [['<sos>', 'va', '!', '<eos>'], ['<sos>', 'marche', '.', '<eos>'], ['<sos>', 'en', 'route', '!', '<eos>'], ['<sos>', 'bouge', '!', '<eos>'], ['<sos>', 'salut', '!', '<eos>']]


In [153]:
#3. text vocabulary ... english 
eng_tokenizer = Tokenizer(filters="", lower=False)           #  Tokenizer를 생성합니다. 
eng_tokenizer.fit_on_texts(encoder)                          # 50000개의 행을 가진 eng의 각 행에 토큰화를 수행
input_text = eng_tokenizer.texts_to_sequences(encoder)       # 단어를 숫자값 인덱스로 변환하여 저장
input_text[:3]

[[28, 1], [28, 1], [28, 1]]

In [154]:
len(input_text)

33000

In [155]:
#3. text vocabulary ... french 
fra_tokenizer = Tokenizer(filters="", lower=False)   #  Tokenizer를 생성합니다. 
fra_tokenizer.fit_on_texts(decoder)                 # 50000개의 행을 가진 fra의 각 행에 토큰화를 수행
target_text = fra_tokenizer.texts_to_sequences(decoder)     # 단어를 숫자값 인덱스로 변환하여 저장
target_text[:3]

[[1, 72, 9, 2], [1, 203, 3, 2], [1, 25, 490, 9, 2]]

In [156]:
#3. text vocabulary ... vocabulary length > 변수 
eng_vocab_size = len(eng_tokenizer.word_index) + 1
fra_vocab_size = len(fra_tokenizer.word_index) + 1
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)

영어 단어장의 크기 : 4672
프랑스어 단어장의 크기 : 8137


In [157]:
#4. 전처리 ... word max len
max_eng_seq_len = max([len(line) for line in input_text])
max_fra_seq_len = max([len(line) for line in target_text])
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

영어 시퀀스의 최대 길이 8
프랑스어 시퀀스의 최대 길이 17


In [158]:
#4. 전처리 ... word max len 확인 
print('전체 샘플의 수 :',len(lines))
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

전체 샘플의 수 : 33000
영어 단어장의 크기 : 4672
프랑스어 단어장의 크기 : 8137
영어 시퀀스의 최대 길이 8
프랑스어 시퀀스의 최대 길이 17


In [159]:
#4. 전처리 ... 모델 decoder 생성
encoder_input = input_text
# 종료 토큰 제거
decoder_input = [[ char for char in line if char != fra_tokenizer.word_index[eos_token] ] for line in target_text] 
# 시작 토큰 제거
decoder_target = [[ char for char in line if char != fra_tokenizer.word_index[sos_token] ] for line in target_text]

In [160]:
#4. 전처리 ... 모델 decoder 생성 확인 
# 디코더의 입력의 경우에는 숫자 12(< eos > 토큰)가 제거되었고, 
#디코더의 출력의 경우에는 숫자 11(< sos > 토큰)이 제거되었습니다.
print(decoder_input[:3])
print(decoder_target[:3])

[[1, 72, 9], [1, 203, 3], [1, 25, 490, 9]]
[[72, 9, 2], [203, 3, 2], [25, 490, 9, 2]]


In [161]:
#4. 전처리 ... padding
encoder_input = pad_sequences(encoder_input, maxlen = max_eng_seq_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen = max_fra_seq_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen = max_fra_seq_len, padding='post')
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (33000, 8)
프랑스어 입력데이터의 크기(shape) : (33000, 17)
프랑스어 출력데이터의 크기(shape) : (33000, 17)


In [162]:
#4. 전처리 ... padding ... 확인(#4. 전처리 ... padding)
print(encoder_input[0])

[28  1  0  0  0  0  0  0]


In [163]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print('랜덤 시퀀스 :',indices)

랜덤 시퀀스 : [18129  4822  1295 ... 30970 15872  2506]


In [164]:
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [165]:
#4. 전처리 ... 33000 중 3000건만 검증데이터로 삼고
n_of_val = 3000

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

print('영어 학습데이터의 크기(shape) :',np.shape(encoder_input_train))
print('프랑스어 학습 입력데이터의 크기(shape) :',np.shape(decoder_input_train))
print('프랑스어 학습 출력데이터의 크기(shape) :',np.shape(decoder_target_train))
print('영어 학습데이터의 크기(shape) :',np.shape(encoder_input_test))
print('프랑스어 학습 입력데이터의 크기(shape) :',np.shape(decoder_input_test))
print('프랑스어 학습 출력데이터의 크기(shape) :',np.shape(decoder_target_test))

영어 학습데이터의 크기(shape) : (30000, 8)
프랑스어 학습 입력데이터의 크기(shape) : (30000, 17)
프랑스어 학습 출력데이터의 크기(shape) : (30000, 17)
영어 학습데이터의 크기(shape) : (3000, 8)
프랑스어 학습 입력데이터의 크기(shape) : (3000, 17)
프랑스어 학습 출력데이터의 크기(shape) : (3000, 17)


In [166]:
# 5. model training ... package loading
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

print('⏳')

⏳


In [167]:
embedding_dim = 64
hidden_units = 64

In [168]:
#5. model training - incoder model ... hidden state , cell state creation 
# 인코더
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(eng_vocab_size, embedding_dim)(encoder_inputs) # 임베딩 층
enc_masking = Masking(mask_value=0.0)(enc_emb) # 패딩 0은 연산에서 제외
encoder_lstm = LSTM(hidden_units, return_state=True) # 상태값 리턴을 위해 return_state는 True
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking) # 은닉 상태와 셀 상태를 리턴
encoder_states = [state_h, state_c] # 인코더의 은닉 상태와 셀 상태를 저장

In [169]:
#5. model training - decoder model
# 디코더
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(fra_vocab_size, hidden_units) # 임베딩 층
dec_emb = dec_emb_layer(decoder_inputs) # 패딩 0은 연산에서 제외
dec_masking = Masking(mask_value=0.0)(dec_emb)

# 상태값 리턴을 위해 return_state는 True, 모든 시점에 대해서 단어를 예측하기 위해 return_sequences는 True
decoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True) 

# 인코더의 은닉 상태를 초기 은닉 상태(initial_state)로 사용
decoder_outputs, _, _ = decoder_lstm(dec_masking,
                                     initial_state=encoder_states)


In [170]:
#  5. model training  - decoder model .. output layer
decoder_softmax_layer = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

In [171]:
#5. model training - model define & compile
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
#model.compile(optimizer="rmsprop", loss="categorical_crossentropy")  기존
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
model.summary()            

Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_18 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
input_19 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_9 (Embedding)         (None, None, 64)     299008      input_18[0][0]                   
__________________________________________________________________________________________________
embedding_10 (Embedding)        (None, None, 64)     520768      input_19[0][0]                   
___________________________________________________________________________________________

In [172]:
#5. model training - model fit
model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, \
          validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size=128, epochs=50)

Epoch 1/50
235/235 [==============================] - 14s 34ms/step - loss: 3.2491 - acc: 0.6342 - val_loss: 1.8840 - val_acc: 0.6428
Epoch 2/50
235/235 [==============================] - 6s 26ms/step - loss: 1.7349 - acc: 0.7106 - val_loss: 1.6333 - val_acc: 0.7556
Epoch 3/50
235/235 [==============================] - 6s 26ms/step - loss: 1.5665 - acc: 0.7564 - val_loss: 1.5196 - val_acc: 0.7607
Epoch 4/50
235/235 [==============================] - 6s 26ms/step - loss: 1.4550 - acc: 0.7691 - val_loss: 1.4204 - val_acc: 0.7753
Epoch 5/50
235/235 [==============================] - 6s 26ms/step - loss: 1.3586 - acc: 0.7793 - val_loss: 1.3347 - val_acc: 0.7874
Epoch 6/50
235/235 [==============================] - 6s 26ms/step - loss: 1.2669 - acc: 0.7964 - val_loss: 1.2503 - val_acc: 0.8066
Epoch 7/50
235/235 [==============================] - 6s 26ms/step - loss: 1.1845 - acc: 0.8109 - val_loss: 1.1843 - val_acc: 0.8164
Epoch 8/50
235/235 [==============================] - 6s 26ms/step -

In [173]:
#6 test model .. encoder_model 
encoder_model = Model(inputs = encoder_inputs, outputs = encoder_states)
encoder_model.summary()

Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_9 (Embedding)      (None, None, 64)          299008    
_________________________________________________________________
masking_8 (Masking)          (None, None, 64)          0         
_________________________________________________________________
lstm_9 (LSTM)                [(None, 64), (None, 64),  33024     
Total params: 332,032
Trainable params: 332,032
Non-trainable params: 0
_________________________________________________________________


In [174]:
#6 test model .. decoder_model 
# 이전 time step의 hidden state를 저장하는 텐서
decoder_state_input_h = Input(shape=(64,))
# 이전 time step의 cell state를 저장하는 텐서
decoder_state_input_c = Input(shape=(64,))
# 이전 time step의 hidden state와 cell state를 하나의 변수에 저장
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
# 훈련 때 사용했던 임베딩 층을 재사용
dec_emb2 = dec_emb_layer(decoder_inputs)

# decoder_states_inputs를 현재 time step의 초기 상태로 사용.
# 구체적인 동작 자체는 def decode_sequence()에 구현.
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state = decoder_states_inputs)

# 현재 time step의 hidden state와 cell state를 하나의 변수에 저장.
decoder_states2 = [state_h2, state_c2]

decoder_outputs2 = decoder_softmax_layer(decoder_outputs2)

In [175]:
#6 test model .. output layer 설계

decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs2] + decoder_states2)
decoder_model.summary()

Model: "model_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_10 (Embedding)        (None, None, 64)     520768      input_19[0][0]                   
__________________________________________________________________________________________________
input_20 (InputLayer)           [(None, 64)]         0                                            
__________________________________________________________________________________________________
input_21 (InputLayer)           [(None, 64)]         0                                            
___________________________________________________________________________________________

In [176]:
#6 test model ... 단어에서 정수로, 정수에서 단어로 바꾸는 사전(dictionary)을 준비
eng2idx = eng_tokenizer.word_index
fra2idx = fra_tokenizer.word_index
idx2eng = eng_tokenizer.index_word
idx2fra = fra_tokenizer.index_word

In [177]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # 에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1,1)) 
    target_seq[0, 0] = fra2idx['<sos>']
    
    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = idx2fra[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += ' '+sampled_char

        # 에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '<eos>' or
           len(decoded_sentence) > max_fra_seq_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장     
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [182]:
#6 test model .. 출력결과 test
import numpy as np
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스 (자유롭게 선택해 보세요)
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    
    print(35 * "-")
    print('입력 문장:', lines.eng[seq_index])
    print('정답 문장:', lines.fra[seq_index][5:len(lines.fra[seq_index])-5]) # '\t'와 '\n'을 빼고 출력
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)]) # '\n'을 빼고 출력

-----------------------------------
입력 문장: Go.
정답 문장:  Bouge ! 
번역기가 번역한 문장:  l endroit est mort
-----------------------------------
입력 문장: Hello!
정답 문장:  Bonjour ! 
번역기가 번역한 문장:  vous avez ete parfait
-----------------------------------
입력 문장: Got it?
정답 문장:  T'as capté ? 
번역기가 번역한 문장:  j ai couru a l etage
-----------------------------------
입력 문장: Hang on.
정답 문장:  Tiens bon ! 
번역기가 번역한 문장:  quelqu un le choix
-----------------------------------
입력 문장: Here's $5.
정답 문장:  Voilà cinq dollars. 
번역기가 번역한 문장:  viens encore dehors
